In [ ]:
#!pip install --user fairseq

In [2]:
import sys
sys.path.append("./fairseq")
sys.path.append("..")

import collections
import math
import os
import random

import torch

from fairseq import checkpoint_utils, distributed_utils, options, progress_bar, tasks, utils
from fairseq.data import iterators
from fairseq.trainer import Trainer
from fairseq.meters import AverageMeter, StopwatchMeter

from node_transformer import node_transformer, node_trainer

print("Torch Version", torch.__version__)

%load_ext autoreload
%autoreload 2

Torch Version 1.1.0


In [16]:
#EXP_ID="transformer_20190703_1800"
#| Generate test with beam=5: BLEU4 = 26.76, 62.4/34.6/21.0/13.1 (BP=0.965, ratio=0.966, syslen=126657, reflen=131161)


#EXP_ID="node_transformer_20190703_2230"
#| Generate test with beam=5: BLEU4 = 22.77, 57.3/29.4/16.6/9.7 (BP=1.000, ratio=1.009, syslen=132351, reflen=131161)


#EXP_ID="transformer_encoder_1layer_20190704_1000"
#| Generate test with beam=5: BLEU4 = 24.76, 60.5/32.2/18.7/11.3 (BP=0.978, ratio=0.978, syslen=128254, reflen=131161)


# EXP_ID="transformer_decoder_1layer_20190710_1530"
# lr = 0.01 epoch 0-27 (nfe infinite at epoch 27) ??? (not sure anymore :D)
# lr = 0.001 epoch 28-...


#EXP_ID="node_transformer_aug_time_dep_20190710_2300"
# encoder 1 layer transformer
# lr = 0.001
# rtol/atol = 0.01
# SUFFIX = "_1"
# SUB_EXP_ID="node_transformer_aug_time_dep_20190710_2300_1"
# lr = 0.001
# adam
# rtol/atol = 0.005


EXP_ID="transformer_encoder_decoder_1layer_20190716_0900"
SUFFIX=""
# encoder/decoder 1 layer transformer
# lr = 0.001

args = [
    #"/home/mandubian/notebooks/pytorch-neural-ode/node-transformer-fair/fairseq/examples/translation/data-bin/wmt14_en_fr",
    "/home/mandubian/notebooks/pytorch-neural-ode/node-transformer-fair/fairseq/examples/translation/data-bin/iwslt14.tokenized.de-en",
    #"--arch", "node_transformer_wmt_en_fr",
    "--arch", "node_transformer",
    #"--arch", "transformer_iwslt_de_en",
    "--task", "translation",
    "--source-lang", "de",
    "--target-lang", "en",
    # NODE PARAMS BEGIN
    #"--node-encoder",
    #"--node-decoder",
    #"--node-rtol", "0.005",
    #"--node-atol", "0.005",
    #"--node-ts", "[0.0, 1.0]",
    #"--node-augment-dims", "1",
    #"--node-time-dependent",
    # NODE PARAMS END    
    "--log-format", "tqdm",
    "--max-tokens", "8000",
    #"--max-sentences", "1000",
    "--num-workers", "8",
    #"--dataset-impl", "lazy",
    "--criterion", "cross_entropy",
    #"--label-smoothing", "0.1",
    "--lr", "0.001",
    #"--lr", "0.0001",
    #"--optimizer", "adam",
    "--save-dir", f"checkpoints/{EXP_ID}{SUFFIX}",
    "--tensorboard-logdir", f"./runs/{EXP_ID}{SUFFIX}",
    "--keep-interval-updates", "10",
    "--restore-file", "checkpoint_best.pt",
    "--keep-last-epochs", "10",
    "--encoder-layers", "1",
    "--decoder-layers", "1",
]
parser = options.get_training_parser()
#add_node_args(parser)
args = options.parse_args_and_arch(parser, input_args=args)
print(args)

Namespace(activation_dropout=0.0, activation_fn='relu', adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, arch='node_transformer', attention_dropout=0.0, bucket_cap_mb=25, clip_norm=25, cpu=False, criterion='cross_entropy', curriculum=0, data='/home/mandubian/notebooks/pytorch-neural-ode/node-transformer-fair/fairseq/examples/translation/data-bin/iwslt14.tokenized.de-en', dataset_impl='cached', ddp_backend='c10d', decoder_attention_heads=8, decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=2048, decoder_input_dim=512, decoder_layers=1, decoder_learned_pos=False, decoder_normalize_before=False, decoder_output_dim=512, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_port=-1, distributed_rank=0, distributed_world_size=1, dropout=0.1, encoder_attention_heads=8, encoder_embed_dim=512, encoder_embed_path=None, encoder_ffn_embed_dim=2048, encoder_layers=1,

In [17]:
if torch.cuda.is_available() and not args.cpu:
    print("using CUDA device", args.device_id)
    torch.cuda.set_device(args.device_id)
_ = torch.manual_seed(args.seed)


using CUDA device 0


In [18]:
# Setup task, e.g., translation, language modeling, etc.
task = tasks.setup_task(args)


| [de] dictionary: 8848 types
| [en] dictionary: 6632 types


In [19]:
# Load valid dataset (we load training data below, based on the latest checkpoint)
for valid_sub_split in args.valid_subset.split(','):
    task.load_dataset(valid_sub_split, combine=True, epoch=0)


| /home/mandubian/notebooks/pytorch-neural-ode/node-transformer-fair/fairseq/examples/translation/data-bin/iwslt14.tokenized.de-en valid de-en 7283 examples


In [20]:

# Build model and criterion
model = task.build_model(args)
criterion = task.build_criterion(args)
print(model)
print('| model {}, criterion {}'.format(args.arch, criterion.__class__.__name__))
print('| num. model params: {} (num. trained: {})'.format(
    sum(p.numel() for p in model.parameters()),
    sum(p.numel() for p in model.parameters() if p.requires_grad),
))

NodeTransformerModel(
  (encoder): TransformerEncoder(
    (embed_tokens): Embedding(8848, 512, padding_idx=1)
    (embed_positions): SinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=512, out_features=512, bias=True)
        )
        (self_attn_layer_norm): LayerNorm(torch.Size([512]), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=512, out_features=2048, bias=True)
        (fc2): Linear(in_features=2048, out_features=512, bias=True)
        (final_layer_norm): LayerNorm(torch.Size([512]), eps=1e-05, elementwise_affine=True)
      )
    )
  )
  (decoder): TransformerDecoder(
    (embed_tokens): Embedding(6632, 512, padding_idx=1)
    (embed_positions): SinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0): TransformerDecoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=512, out_fea

In [21]:
# Build trainer
trainer = node_trainer.Trainer(args, task, model, criterion)
print('| training on {} GPUs'.format(args.distributed_world_size))
print('| max tokens per GPU = {} and max sentences per GPU = {}'.format(
    args.max_tokens,
    args.max_sentences,
))

| training on 1 GPUs
| max tokens per GPU = 8000 and max sentences per GPU = None


In [25]:
## Load the latest checkpoint if one is available and restore the
# corresponding train iterator
save_dir_keep = args.save_dir
args.save_dir = f"checkpoints/{EXP_ID}"
args.reset_optimizer = True
extra_state, epoch_itr = checkpoint_utils.load_checkpoint(args, trainer)
args.save_dir = save_dir_keep
args.reset_optimizer = False


| no existing checkpoint found checkpoints/transformer_encoder_decoder_1layer_20190716_0900/checkpoint_best.pt
| loading train data for epoch 0
| /home/mandubian/notebooks/pytorch-neural-ode/node-transformer-fair/fairseq/examples/translation/data-bin/iwslt14.tokenized.de-en train de-en 160239 examples


In [ ]:
from train import validate

def train(args, trainer, task, epoch_itr):
    """Train the model for one epoch."""
    # Update parameters every N batches
    update_freq = args.update_freq[epoch_itr.epoch - 1] \
        if epoch_itr.epoch <= len(args.update_freq) else args.update_freq[-1]

    # Initialize data iterator
    itr = epoch_itr.next_epoch_itr(
        fix_batches_to_gpus=args.fix_batches_to_gpus,
        shuffle=(epoch_itr.epoch >= args.curriculum),
    )
    itr = iterators.GroupedIterator(itr, update_freq)
    progress = progress_bar.build_progress_bar(
        args, itr, epoch_itr.epoch, no_progress_bar='simple',
    )

    extra_meters = collections.defaultdict(lambda: AverageMeter())
    valid_subsets = args.valid_subset.split(',')
    max_update = args.max_update or math.inf
    
    #timesteps = torch.FloatTensor(args.ts).to()
    for i, samples in enumerate(progress, start=epoch_itr.iterations_in_epoch):
        log_output = trainer.train_step(samples)
        if log_output is None:
            continue

        # log mid-epoch stats
        stats = get_training_stats(trainer)
        for k, v in log_output.items():
            if k in ['loss', 'nll_loss', 'ntokens', 'nsentences', 'sample_size']:
                continue  # these are already logged above
            if 'loss' in k:
                extra_meters[k].update(v, log_output['sample_size'])
            else:
                extra_meters[k].update(v)
            stats[k] = extra_meters[k].avg
        progress.log(stats, tag='train', step=stats['num_updates'])

        # ignore the first mini-batch in words-per-second calculation
        if i == 0:
            trainer.get_meter('wps').reset()

        num_updates = trainer.get_num_updates()
        if (
            not args.disable_validation
            and args.save_interval_updates > 0
            and num_updates % args.save_interval_updates == 0
            and num_updates > 0
        ):
            valid_losses = validate(args, trainer, task, epoch_itr, valid_subsets)
            checkpoint_utils.save_checkpoint(args, trainer, epoch_itr, valid_losses[0])

        if num_updates >= max_update:
            break

    # log end-of-epoch stats
    stats = get_training_stats(trainer)
    for k, meter in extra_meters.items():
        stats[k] = meter.avg
    progress.print(stats, tag='train', step=stats['num_updates'])

    # reset training meters
    for k in [
        'train_loss', 'train_nll_loss', 'wps', 'ups', 'wpb', 'bsz', 'gnorm', 'clip',
    ]:
        meter = trainer.get_meter(k)
        if meter is not None:
            meter.reset()


def get_training_stats(trainer):
    stats = collections.OrderedDict()
    stats['loss'] = trainer.get_meter('train_loss')
    if trainer.get_meter('train_nll_loss').count > 0:
        nll_loss = trainer.get_meter('train_nll_loss')
        stats['nll_loss'] = nll_loss
    else:
        nll_loss = trainer.get_meter('train_loss')
    stats['ppl'] = utils.get_perplexity(nll_loss.avg)
    stats['wps'] = trainer.get_meter('wps')
    stats['ups'] = trainer.get_meter('ups')
    stats['wpb'] = trainer.get_meter('wpb')
    stats['bsz'] = trainer.get_meter('bsz')
    stats['num_updates'] = trainer.get_num_updates()
    stats['lr'] = trainer.get_lr()
    stats['gnorm'] = trainer.get_meter('gnorm')
    stats['clip'] = trainer.get_meter('clip')
    stats['oom'] = trainer.get_meter('oom')
    if trainer.get_meter('loss_scale') is not None:
        stats['loss_scale'] = trainer.get_meter('loss_scale')
    stats['wall'] = round(trainer.get_meter('wall').elapsed_time)
    stats['train_wall'] = trainer.get_meter('train_wall')
    if trainer._model.has_node_encoder:
        stats['nfe_encoder'] = trainer.get_meter('nfe_encoder')
    if trainer._model.has_node_decoder:
        stats['nfe_decoder'] = trainer.get_meter('nfe_decoder')
    return stats



# Train until the learning rate gets too small
max_epoch = args.max_epoch or math.inf
max_update = args.max_update or math.inf
lr = trainer.get_lr()
train_meter = StopwatchMeter()
train_meter.start()
valid_losses = [None]
valid_subsets = args.valid_subset.split(',')
while lr > args.min_lr and epoch_itr.epoch < max_epoch and trainer.get_num_updates() < max_update:
    # train for one epoch
    train(args, trainer, task, epoch_itr)

    if not args.disable_validation and epoch_itr.epoch % args.validate_interval == 0:
        valid_losses = validate(args, trainer, task, epoch_itr, valid_subsets)
    else:
        valid_losses = [None]

    # only use first validation loss to update the learning rate
    lr = trainer.lr_step(epoch_itr.epoch, valid_losses[0])

    # save checkpoint
    if epoch_itr.epoch % args.save_interval == 0:
        checkpoint_utils.save_checkpoint(args, trainer, epoch_itr, valid_losses[0])

    if ':' in getattr(args, 'data', ''):
        # sharded data: get train iterator for next epoch
        epoch_itr = trainer.get_train_iterator(epoch_itr.epoch)
train_meter.stop()
print('| done training in {:.1f} seconds'.format(train_meter.sum))


| epoch 045 | loss 5.643 | ppl 49.98 | wps 1556 | ups 0 | wpb 13386.827 | bsz 543.183 | num_updates 295 | lr 0.001 | gnorm 1.934 | clip 0.003 | oom 0.000 | wall 2557 | train_wall 104536 | nfe_decoder 119.920
| epoch 045 | valid on 'valid' subset | loss 4.997 | ppl 31.94 | num_updates 295
| saved checkpoint checkpoints/node_transformer_aug_time_dep_20190710_2300/checkpoint45.pt (epoch 45 @ 295 updates) (writing took 0.4802522659301758 seconds)
| epoch 046 | loss 4.670 | ppl 25.45 | wps 1272 | ups 0 | wpb 13386.827 | bsz 543.183 | num_updates 590 | lr 0.001 | gnorm 1.264 | clip 0.000 | oom 0.000 | wall 5678 | train_wall 107642 | nfe_decoder 120.795
| epoch 046 | valid on 'valid' subset | loss 4.670 | ppl 25.46 | num_updates 590 | best_loss 4.67031
| saved checkpoint checkpoints/node_transformer_aug_time_dep_20190710_2300/checkpoint46.pt (epoch 46 @ 590 updates) (writing took 0.4254164695739746 seconds)
| epoch 047 | loss 4.400 | ppl 21.12 | wps 1128 | ups 0 | wpb 13386.827 | bsz 543.183 